In [1]:
### Buliding Our Custom Multilingual Tokenizer
from collections import Counter
import json
import pandas as pd

In [2]:
# full_data = pd.read_csv('/home/IITB/ai-at-ieor/23m1508/Shivam_23M1508/Interspeech/data/set1_set2_set3_combined.csv')


full_data = pd.read_csv('/home/IITB/ai-at-ieor/23m1508/Shivam_23M1508/Interspeech/data/scratch_balanced_shuffled_data.csv')

In [4]:
full_data['class'].value_counts()

class
english       428641
korean        110004
japanese      107183
italian       105837
russian        97721
portuguese     95924
spanish        94066
vietnamese     93151
french         91711
thai           87276
german         87060
Name: count, dtype: int64

In [7]:
eng_data = full_data[full_data['class'] == 'english']

In [12]:
eng_data.shape

(428641, 4)

In [13]:
kor_data = full_data[full_data['class'] == 'korean']

In [15]:
kor_data.shape

(110004, 4)

In [16]:
jap_data = full_data[full_data['class'] == 'japanese']

In [19]:
jap_data.shape

(107183, 4)

In [20]:
ita_data = full_data[full_data['class'] == 'italian']

In [23]:
ita_data.shape

(105837, 4)

In [24]:
rus_data = full_data[full_data['class'] == 'russian']

In [27]:
rus_data.shape

(97721, 4)

In [28]:
por_data = full_data[full_data['class'] == 'portuguese']

In [31]:
por_data.shape

(95924, 4)

In [32]:
spa_data = full_data[full_data['class'] == 'spanish']

In [34]:
spa_data.shape

(94066, 4)

In [35]:
vie_data = full_data[full_data['class'] == 'vietnamese']

In [37]:
vie_data.shape

(93151, 4)

In [38]:
fre_data = full_data[full_data['class'] == 'french']

In [40]:
fre_data.shape

(91711, 4)

In [41]:
tha_data = full_data[full_data['class'] == 'thai']

In [44]:
tha_data.shape

(87276, 4)

In [45]:
ger_data = full_data[full_data['class'] == 'german']

In [47]:
ger_data.shape

(87060, 4)

In [44]:
# full_data.loc[full_data['class'] == 'english', 'Text'] = full_data.loc[full_data['class'] == 'english', 'Text'].apply(lambda x: x.lower())

In [117]:
# def lowercase_english_letters(text):
#     return ''.join([c.lower() if c.isascii() and c.isalpha() else c for c in text])

In [156]:
def lowercase_all_letters(text):
    return ''.join([c.lower() if c.isalpha() else c for c in text])

In [157]:
ger_data['Text'] = ger_data['Text'].apply(lowercase_all_letters)

/tmp/ipykernel_4050649/2431544936.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ger_data['Text'] = ger_data['Text'].apply(lowercase_all_letters)


In [120]:
rus_data.iloc[84453]['Text']

'но, я думаю,'

In [159]:
import pandas as pd
from collections import Counter
import json
import re


def clean_text(text):
    characters_to_remove = r'[,\?\.\!\-\;\:\%\'\`\{}()@#$%^&*\+\[\]\_｡]+'
    cleaned_text = re.sub(characters_to_remove, '', text)
    # cleaned_text = ''.join([c.lower() if c.isascii() else c for c in cleaned_text]) + ' '
    return cleaned_text

# def clean_text(text):

#     characters_to_remove = r'[,\?\.\!\-\;\:\%\'\`\{}()@#$%^&*\+\[\]\_]+'
#     cleaned_text = re.sub(characters_to_remove, '', text).upper() + ' '
#     return cleaned_text


def build_vocab_from_dataframe(df, text_column, output_vocab_path, min_freq = 10):

    # Replace spaces with '|' for CTC-friendly word boundary
    text_data = df[text_column].astype(str).str.replace(" ", "|", regex = False).str.cat(sep = "")

    text_data = clean_text(text_data)
    
    # Create character list
    chars = list(text_data)

    # Count characters
    counter = Counter(chars)

    filtered_items = [(char, count) for char, count in counter.items() if count >= min_freq]
    
    # Build vocab: sorted by character
    vocab = {char: i for i, (char, _) in enumerate(sorted(filtered_items, key=lambda x: x[0]))}

    # Add CTC or model-specific special tokens
    special_tokens = ["<pad>", "<unk>", "<s>", "</s>"]
    for token in special_tokens:
        if token not in vocab:
            vocab[token] = len(vocab)

    # Save to JSON
    with open(output_vocab_path, "w", encoding="utf-8") as f:
        json.dump(vocab, f, ensure_ascii = False, indent = 2)

    print(f"Vocab saved to {output_vocab_path} with {len(vocab)} tokens (min freq = {min_freq}).")

In [160]:
build_vocab_from_dataframe(
                                df=ger_data,
                                text_column="Text",
                                output_vocab_path="/home/IITB/ai-at-ieor/23m1508/Shivam_23M1508/Interspeech/code/model_code/model_files/ger_vocab.json",
                                min_freq = 10
                            )

Vocab saved to /home/IITB/ai-at-ieor/23m1508/Shivam_23M1508/Interspeech/code/model_code/model_files/ger_vocab.json with 36 tokens (min freq = 10).


In [15]:
# Vocab saved to /home/IITB/ai-at-ieor/23m1508/Shivam_23M1508/Interspeech/code/model_code/model_files/multilingual_vocab.json with 3086 tokens (min freq = 10).

In [62]:
kor_data

,ID,Text,Path,class
2,0003-0003_005-O2-049763-050186,다은이 나오고 그날 유미랑 만나는 거 다은이랑 유미랑 만나는 거.,/scratch/IITB/ai-at-ieor/23m1508/23m1508_backu...,korean
5,0210-0210_003-O1-006979-007581,근데 이게 또 이게 교육이라는 게 참 가르치는 사람이 진짜 중요하잖아요.,/scratch/IITB/ai-at-ieor/23m1508/23m1508_backu...,korean
14,0191-0191_003-O2-053992-054564,아무런 선택지가 없이 겪어야 되는 게 제일 힘든 점이라고 생각을 해.,/scratch/IITB/ai-at-ieor/23m1508/23m1508_backu...,korean
28,0188-0188_003-O2-052166-052517,어 처음 들어보는것 같아요. 맛이 어떤가요 비싸진 않나요?,/scratch/IITB/ai-at-ieor/23m1508/23m1508_backu...,korean
35,0135-0135_005-O2-114136-114226,정신 파예요?,/scratch/IITB/ai-at-ieor/23m1508/23m1508_backu...,korean
...,...,...,...,...
1199961,0204-0204_004-O2-023268-023637,그럼 나 두시에 일어나 세시에 일어나 네시에 일어나 이래 근데 자기는 자.,/scratch/IITB/ai-at-ieor/23m1508/23m1508_backu...,korean
1199972,0185-0185_001-O2-006845-007004,그 정호연인가?,/scratch/IITB/ai-at-ieor/23m1508/23m1508_backu...,korean
1199987,0137-0137_005-O2-051707-052314,물고기들이 이렇게 지나가면은 이제 그 지나가는게 눈에 다 보일 정도로 그런 깨끗한 ...,/scratch/IITB/ai-at-ieor/23m1508/23m1508_backu...,korean
1199989,0188-0188_004-O2-064345-064575,아무리 먹어도 그 칼로리,/scratch/IITB/ai-at-ieor/23m1508/23m1508_backu...,korean


### Adding the blank token in each vocabulary

In [2]:
import json

In [1]:
# main the order as per the lang2id
vocab_paths = {
                "english" : "/home/IITB/ai-at-ieor/23m1508/Shivam_23M1508/Interspeech/code/model_code/model_files/eng_vocab.json",
                "korean" : "/home/IITB/ai-at-ieor/23m1508/Shivam_23M1508/Interspeech/code/model_code/model_files/kor_vocab.json",
                "japanese" : "/home/IITB/ai-at-ieor/23m1508/Shivam_23M1508/Interspeech/code/model_code/model_files/jap_vocab.json",
                "italian" : "/home/IITB/ai-at-ieor/23m1508/Shivam_23M1508/Interspeech/code/model_code/model_files/ita_vocab.json",
                "russian" : "/home/IITB/ai-at-ieor/23m1508/Shivam_23M1508/Interspeech/code/model_code/model_files/rus_vocab.json",
                "portuguese" : "/home/IITB/ai-at-ieor/23m1508/Shivam_23M1508/Interspeech/code/model_code/model_files/por_vocab.json",
                "spanish" : "/home/IITB/ai-at-ieor/23m1508/Shivam_23M1508/Interspeech/code/model_code/model_files/spa_vocab.json",
                "vietnamese" : "/home/IITB/ai-at-ieor/23m1508/Shivam_23M1508/Interspeech/code/model_code/model_files/vie_vocab.json",
                "french" : "/home/IITB/ai-at-ieor/23m1508/Shivam_23M1508/Interspeech/code/model_code/model_files/fre_vocab.json",
                "thai" : "/home/IITB/ai-at-ieor/23m1508/Shivam_23M1508/Interspeech/code/model_code/model_files/tha_vocab.json",
                "german" : "/home/IITB/ai-at-ieor/23m1508/Shivam_23M1508/Interspeech/code/model_code/model_files/ger_vocab.json"
                }

In [ ]:

# Load, modify, and overwrite vocab files
for lang, path in vocab_paths.items():
    with open(path, "r", encoding="utf-8") as f:
        vocab = json.load(f)

    # Sort vocab by value (index) to ensure order, then re-insert <blank> at 0
    sorted_vocab = sorted(vocab.items(), key = lambda x: x[1])
    tokens = [token for token, _ in sorted_vocab]

    if "<blank>" not in tokens:
        tokens = ["<blank>"] + tokens
    elif tokens[0] != "<blank>":
        tokens.remove("<blank>")
        tokens = ["<blank>"] + tokens

    new_vocab = {token: idx for idx, token in enumerate(tokens)}

    with open(path, "w", encoding="utf-8") as f:
        json.dump(new_vocab, f, indent=2, ensure_ascii=False)


In [ ]:
vocab_paths.keys()  # Return the languages updated

dict_keys(['english', 'korean', 'japanese', 'italian', 'russian', 'portuguese', 'spanish', 'vietnamese', 'french', 'thai', 'german'])